# 使用ResNet50进行训练
可视化结果

In [18]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
import cv2
import numpy as np
import h5py
import os
 
from keras.utils import np_utils, conv_utils
#from keras.utils.generic_utils import unpack_singleton
from keras.models import Model
#from keras.layers import Flatten, Dense, Input 
from keras.optimizers import Adam
#预训练模型
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet201

from keras import backend as K
#from past.builtins import xrange
from keras.layers import AveragePooling2D,GlobalAveragePooling2D
from keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, concatenate, \
    Activation, ZeroPadding2D, Flatten
from keras.layers import add 
from keras.utils import plot_model
from keras.metrics import top_k_categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

Width = 224
Height = 224
num_classes = 71 #71  #102       #Caltech101为102  cifar10为10
pic_dir_out = './pic_out/'#'./pic_out2/'  
pic_dir_data = './pic_dataset/'#'./pic_dataset2/'
sub_dir = '224_resnet50/'
batch_size=30
pic_dir_mine = os.path.join(pic_dir_out,sub_dir)

train_datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    rescale=1./255,
    data_format = 'channels_last',
    validation_split=0.3
)
train_generator = train_datagen.flow_from_directory(
    pic_dir_data,#+'train',
    target_size=(Width, Height),
    batch_size=batch_size,
    shuffle=True,
    seed=0,
    subset="training"
)

#valid_datagen = ImageDataGenerator( 
    #width_shift_range=0.1,
    #height_shift_range=0.1,
    #shear_range=0.1,
    #zoom_range=0.1,
    #horizontal_flip=True,
    #rescale=1./255，
    #data_format = 'channels_last',
#)
vaild_generator = train_datagen.flow_from_directory(
    pic_dir_data,#+'val',
    target_size=(Width, Height),
    batch_size=batch_size,
    shuffle=True,
    seed=0,
    subset="validation"
)

Found 15526 images belonging to 71 classes.
Found 6615 images belonging to 71 classes.


In [19]:
import matplotlib.pyplot as plt
%matplotlib inline
def accuracy_curve(h):
    acc, loss, val_acc, val_loss = h.history['acc'], h.history['loss'], h.history['val_acc'], h.history['val_loss']
    epoch = len(acc)
    plt.figure(figsize=(17, 5))
    plt.subplot(121)
    plt.plot(range(epoch), acc, label='Train')
    plt.plot(range(epoch), val_acc, label='Test')
    plt.title('Accuracy over ' + str(epoch) + ' Epochs', size=15)
    plt.legend()
    plt.grid(True)
    plt.subplot(122)
    plt.plot(range(epoch), loss, label='Train')
    plt.plot(range(epoch), val_loss, label='Test')
    plt.title('Loss over ' + str(epoch) + ' Epochs', size=15)
    plt.legend()
    plt.grid(True)
    plt.show()

In [20]:
#使用imagenet预训练的模型参数
#不适用数据增强
input_tensor = Input(shape=(Width, Height, 3),name='image_input')
base_model = ResNet50(input_tensor=input_tensor,include_top=False,weights='imagenet')
#base_model = InceptionResNetV2(input_tensor=input_tensor,include_top=False,weights='imagenet')#,pooling = 'avg')
#base_model = VGG19(input_tensor=input_tensor,include_top=False,weights='imagenet')#,pooling = 'avg')
#base_model = InceptionV3(input_tensor=input_tensor,include_top=False,weights='imagenet')#,pooling = 'avg')
#base_model = DenseNet201(input_tensor=input_tensor,include_top=False,weights='imagenet')#,pooling = 'avg')#'imagenet'
# get_resnet50_output = K.function([base_model.layers[0].input, K.learning_phase()],
#                           [base_model.layers[-1].output])
out_bm=base_model.output
pooling=AveragePooling2D((7, 7), name='avg_pool')(out_bm)
fl = Flatten(name='flatten')(pooling)
dense = Dense(1024, activation='relu', name='fc1')(fl)
#drop = Dropout(0.5, name='drop')(dense)
pred = Dense(num_classes, activation='softmax', name='predictions')(dense)

fine_model = Model(inputs=input_tensor,outputs=pred)
for layer in base_model.layers:
    layer.trainable = False
    #print (layer)

'''
ResNet50输出尺寸(7, 7, 2048)     epoch30
DenseNet201输出尺寸(7,7,1920)    epoch50
InceptionResNetV2输出尺寸(5,5,1536)不行的
VGG19输出尺寸(7,7,512)
InceptionV3输出尺寸(5,5,2048)
'''
es = EarlyStopping(monitor='val_acc', patience=20)

fine_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
#fine_model.summary()
h=fine_model.fit_generator(train_generator,validation_data=vaild_generator,epochs=50,steps_per_epoch=train_generator.n//batch_size,\
                           validation_steps=vaild_generator.n//batch_size,verbose=1,callbacks=[es])
#model.fit(resnet50_train_output, y_train, epochs=100, batch_size=64,) #batch_size有没有影响
#model.save_weights(os.path.join(pic_dir_mine,'cnn_model_Kaggle_resnet50_'+cm2_str+'.h5'))

#画图
accuracy_curve(h)

C:\Anaconda\envs\t\lib\site-packages\keras_applications\resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/50
485/485 [==============================] - 405s 835ms/step - loss: 2.3625 - acc: 0.4029 - val_loss: 5.5729 - val_acc: 0.0181
Epoch 2/50
485/485 [==============================] - 318s 655ms/step - loss: 1.5847 - acc: 0.5685 - val_loss: 6.7497 - val_acc: 0.0138
Epoch 3/50
485/485 [==============================] - 318s 656ms/step - loss: 1.3271 - acc: 0.6277 - val_loss: 8.3422 - val_acc: 0.0138
Epoch 4/50
485/485 [==============================] - 309s 636ms/step - loss: 1.1367 - acc: 0.6785 - val_loss: 9.8272 - val_acc: 0.0181
Epoch 5/50
485/485 [==============================] - 308s 636ms/step - loss: 0.9924 - acc: 0.7094 - val_loss: 12.0137 - val_acc: 0.0181
Epoch 6/50
440/485 [==========================>...] - ETA: 19s - loss: 0.8968 - acc: 0.7391

KeyboardInterrupt: 

In [22]:
#使用一般的ResNet
#不适用数据增强
def Conv2d_BN(x, nb_filter, kernel_size, strides=(1, 1), padding='same', name=None):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None

    x = Conv2D(nb_filter, kernel_size, padding=padding, strides=strides, activation='relu', name=conv_name,kernel_initializer='he_normal')(x)
    x = BatchNormalization(axis=3, name=bn_name)(x)
    return x

def identity_Block(inpt, nb_filter, kernel_size, strides=(1, 1), with_conv_shortcut=False):
    x = Conv2d_BN(inpt, nb_filter=nb_filter, kernel_size=kernel_size, strides=strides, padding='same')
    x = Conv2d_BN(x, nb_filter=nb_filter, kernel_size=kernel_size, padding='same')
    if with_conv_shortcut:
        shortcut = Conv2d_BN(inpt, nb_filter=nb_filter, strides=strides, kernel_size=kernel_size)
        x = add([x, shortcut])
        return x
    else:
        x = add([x, inpt])
        return x

def bottleneck_Block(inpt,nb_filters,strides=(1,1),with_conv_shortcut=False):
    k1,k2,k3=nb_filters
    x = Conv2d_BN(inpt, nb_filter=k1, kernel_size=1, strides=strides, padding='same')
    x = Conv2d_BN(x, nb_filter=k2, kernel_size=3, padding='same')
    x = Conv2d_BN(x, nb_filter=k3, kernel_size=1, padding='same')
    if with_conv_shortcut:
        shortcut = Conv2d_BN(inpt, nb_filter=k3, strides=strides, kernel_size=1)
        x = add([x, shortcut])
        return x
    else:
        x = add([x, inpt])
        return x

def resnet_50(width,height,channel,classes):
    inpt = Input(shape=(width, height, channel))
    x = ZeroPadding2D((3, 3))(inpt)
    x = Conv2d_BN(x, nb_filter=64, kernel_size=(7, 7), strides=(2, 2), padding='valid')
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    #conv2_x
    x = bottleneck_Block(x, nb_filters=[64,64,256],strides=(1,1),with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[64,64,256])
    x = bottleneck_Block(x, nb_filters=[64,64,256])

    #conv3_x
    x = bottleneck_Block(x, nb_filters=[128, 128, 512],strides=(2,2),with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[128, 128, 512])
    x = bottleneck_Block(x, nb_filters=[128, 128, 512])
    x = bottleneck_Block(x, nb_filters=[128, 128, 512])

    #conv4_x
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024],strides=(2,2),with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])

    #conv5_x
    x = bottleneck_Block(x, nb_filters=[512, 512, 2048], strides=(2, 2), with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[512, 512, 2048])
    x = bottleneck_Block(x, nb_filters=[512, 512, 2048])

    x = AveragePooling2D(pool_size=(7, 7))(x)
    x = Flatten()(x)
    x = Dense(classes, activation='softmax',kernel_initializer='he_normal')(x)

    model = Model(inputs=inpt, outputs=x)
    return model

es = EarlyStopping(monitor='val_acc', patience=20)
model = resnet_50(Width,Height,3,num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
h = model.fit_generator(train_generator,validation_data=vaild_generator,epochs=50,steps_per_epoch=train_generator.n//batch_size,\
                           validation_steps=vaild_generator.n//batch_size,verbose=1,callbacks=[es])
#画图
accuracy_curve(h)

Epoch 1/50


ResourceExhaustedError: OOM when allocating tensor with shape[32,56,56,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_10/Adam/gradients/zeros_181-0-1-TransposeNCHWToNHWC-LayoutOptimizer = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_10/Adam/gradients/zeros_181, PermConstNCHWToNHWC-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: metrics_11/acc/Mean/_28021 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_19019_metrics_11/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add repo

In [5]:
def Conv2d_BN(x, nb_filter, kernel_size, strides=(1, 1), padding='same', name=None):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None

    x = Conv2D(nb_filter, kernel_size, padding=padding, strides=strides, activation='relu', name=conv_name,kernel_initializer='he_normal')(x)
    x = BatchNormalization(axis=3, name=bn_name)(x)
    return x

def identity_Block(inpt, nb_filter, kernel_size, strides=(1, 1), with_conv_shortcut=False):
    x = Conv2d_BN(inpt, nb_filter=nb_filter, kernel_size=kernel_size, strides=strides, padding='same')
    x = Conv2d_BN(x, nb_filter=nb_filter, kernel_size=kernel_size, padding='same')
    if with_conv_shortcut:
        shortcut = Conv2d_BN(inpt, nb_filter=nb_filter, strides=strides, kernel_size=kernel_size)
        x = add([x, shortcut])
        return x
    else:
        x = add([x, inpt])
        return x

def bottleneck_Block(inpt,nb_filters,strides=(1,1),with_conv_shortcut=False):
    k1,k2,k3=nb_filters
    x = Conv2d_BN(inpt, nb_filter=k1, kernel_size=1, strides=strides, padding='same')
    x = Conv2d_BN(x, nb_filter=k2, kernel_size=3, padding='same')
    x = Conv2d_BN(x, nb_filter=k3, kernel_size=1, padding='same')
    if with_conv_shortcut:
        shortcut = Conv2d_BN(inpt, nb_filter=k3, strides=strides, kernel_size=1)
        x = add([x, shortcut])
        return x
    else:
        x = add([x, inpt])
        return x

def resnet_50(width,height,channel,classes):
    inpt = Input(shape=(width, height, channel))
    x = ZeroPadding2D((3, 3))(inpt)
    x = Conv2d_BN(x, nb_filter=64, kernel_size=(7, 7), strides=(2, 2), padding='valid')
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    #conv2_x
    x = bottleneck_Block(x, nb_filters=[64,64,256],strides=(1,1),with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[64,64,256])
    x = bottleneck_Block(x, nb_filters=[64,64,256])

    #conv3_x
    x = bottleneck_Block(x, nb_filters=[128, 128, 512],strides=(2,2),with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[128, 128, 512])
    x = bottleneck_Block(x, nb_filters=[128, 128, 512])
    x = bottleneck_Block(x, nb_filters=[128, 128, 512])

    #conv4_x
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024],strides=(2,2),with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])

    #conv5_x
    x = bottleneck_Block(x, nb_filters=[512, 512, 2048], strides=(2, 2), with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[512, 512, 2048])
    x = bottleneck_Block(x, nb_filters=[512, 512, 2048])

    x = AveragePooling2D(pool_size=(7, 7))(x)
    x = Flatten()(x)
    x = Dense(classes, activation='softmax',kernel_initializer='he_normal')(x)

    model = Model(inputs=inpt, outputs=x)
    return model

#使用一般的ResNet
#不适用数据增强
es = EarlyStopping(monitor='val_acc', patience=20)
model = resnet_50(Width,Height,3,num_classes)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
h=model.fit_generator(train_generator,validation_data=vaild_generator,epochs=50,steps_per_epoch=train_generator.n//batch_size,
                           validation_steps=vaild_generator.n//batch_size,verbose=1,callbacks=[es])
#画图
accuracy_curve(h)

NameError: name 'fine_model' is not defined